# Excellent explanation of optimising a function with Cython:
[https://flothesof.github.io/cython-complex-numbers-parallel-optimization.html](https://flothesof.github.io/cython-complex-numbers-parallel-optimization.html)

In [71]:
import numpy as np
import numpy.typing as npt

%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [72]:
rng = np.random.default_rng(0)
M = 40
args = 0, M, rng.uniform(low=0.1, high=10, size=M), rng.uniform(low=1, high=1e3, size=M) * 1e-9, 2*np.pi/(2000*1e-9), 2, 3, -np.pi/7

In [73]:
def test_matrices_close(m1: np.ndarray, m2: np.ndarray):
    assert m1.shape[0] == m2.shape[0] and m1.shape[1] == m2.shape[1]
    np.testing.assert_almost_equal(m1, m2)
    return True

In [74]:
def profile(func):
    rng = np.random.default_rng(0)
    num = 3
    for M in range(1, 40):
        for n in (rng.uniform(low=0.1, high=10, size=M) for _ in range(num)):
            for d in (rng.uniform(low=1, high=1e3, size=M) * 1e-9 for _ in range(num)):
                for wavelength in np.linspace(200, 3000, num=num) * 1e-9:
                    for n_outer in np.linspace(0.1, 10, num=num):
                        for n_substrate in np.linspace(0.1, 10, num=num):
                            for theta_outer in np.linspace(-np.pi / 2 + 1e-3, np.pi / 2 - 1e-3, num=num):
                                for polarisation in 0, 1:
                                    args = polarisation, M, n, d, wavelength, n_outer, n_substrate, theta_outer
                                    func(*args)

# Native Python and NumPy



In [75]:
def _make_matrix(polarisation: int, M: np.int_, n: npt.NDArray[np.float_], d: npt.NDArray[np.float_], k_outer: np.float_, n_outer: np.float_, n_substrate: np.float_,
                 theta_outer: np.float_) -> npt.NDArray[np.complex_]:
    """
    Constructs the matrix \mathbf{M} in band structure form.

    @param polarisation: 0 for s-polarisation and 1 for p-polarisation
    @param M: Number of layers. Valid input range: M ≥ 1
    @param n: Array storing the refractive indices of the layers.
    @param d: Array storing the thicknesses of the layers.
    @param k_outer: Wavenumber "2*pi/lambda" of the incident light in the outer medium. lambda is wavelength in the
    outer medium
    @param n_outer: Refractive index of the outer medium.
    @param n_substrate: Refractive index of the substrate.
    @param theta_outer: Angle in the outer medium, i.e. incident angle. Valid range: -pi/2 < theta_outer < pi/2
    @return: Matrix \mathbf{M}
    """
    # assert len(n) == M and len(d) == M

    # Concatenate arrays to call np.sqrt only once, which increases performance.
    all_n = np.concatenate([np.array([n_outer]), n, np.array([n_substrate])])
    all_cos_theta = np.sqrt(np.complex_(1 - (n_outer / all_n) ** 2 * np.sin(theta_outer) ** 2))

    cos_theta_outer: np.complex_ = all_cos_theta[0]
    cos_theta: npt.NDArray[np.complex_] = all_cos_theta[1:M+1]
    cos_theta_substrate: np.complex_ = all_cos_theta[M+1]

    # Pre-compute complex exponentials for performance increase.
    exps: npt.NDArray[np.complex_] = np.exp(1j * (k_outer * d) * (n / n_outer) * cos_theta)

    (a, b, c, d, columns) = _make_s_pol_submatrices(M, exps, cos_theta, cos_theta_outer, cos_theta_substrate, n, n_outer, n_substrate)

    mat: npt.NDArray[np.complex_] = np.hstack([
        np.array([[0], [0], [a], [b], [0]], dtype=np.complex_),
        *columns,
        np.array([[0], [c], [d], [0], [0]], dtype=np.complex_)
    ])

    #assert mat.shape[0] == 5 and mat.shape[1] == 2 * M + 2
    return mat


def _make_s_pol_submatrices(M: np.int_, exps: npt.NDArray[np.complex_], cos_theta: npt.NDArray[np.complex_],
                            cos_theta_outer: np.complex_, cos_theta_substrate: np.complex_, n: npt.NDArray[np.float_],
                            n_outer: np.float_, n_substrate: np.float_) \
        -> tuple[np.complex_, np.complex_, np.complex_, np.complex_, list[npt.NDArray[np.complex_]]]:
    a: np.complex_ = np.complex_(-1)
    b: np.complex_ = np.complex_(1)
    c: np.complex_ = np.complex_(1)
    d: np.complex_ = np.complex_(n_substrate * cos_theta_substrate)

    columns: list[npt.NDArray[np.complex_]] = [
        np.array([
                [0, exps[0]],
                [1, -(n[0] / n_outer) * (cos_theta[0] / cos_theta_outer) * exps[0]],
                [(n[0] / n_outer) * (cos_theta[0] / cos_theta_outer), -1],
                [-exps[0], n[0] * cos_theta[0]],
                [-exps[0] * n[0] * cos_theta[0], 0]
            ], dtype=np.complex_)
        ] + [
            np.array([
                [0, exps[j]],
                [1, -n[j] * cos_theta[j] * exps[j]],
                [n[j] * cos_theta[j], -1],
                [-exps[j], n[j] * cos_theta[j]],
                [-exps[j] * n[j] * cos_theta[j], 0]
            ])
            for j in range(1, M)
        ]

    return (a, b, c, d, columns)

In [76]:
%timeit _make_matrix(*args)

294 µs ± 176 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Transcribing _make_matrix() to Cython


In [77]:
%%cython -a

cimport numpy as np
import numpy as np

cpdef np.ndarray [complex, ndim=2] make_matrix_cython(int polarisation, int M, np.ndarray [double, ndim=1] n, np.ndarray [double, ndim=1] d, double k_outer, double n_outer, double n_substrate, double theta_outer):
    all_n = np.concatenate([np.array([n_outer]), n, np.array([n_substrate])])
    all_cos_theta = np.sqrt(np.complex_(1 - (n_outer / all_n) ** 2 * np.sin(theta_outer) ** 2))

    cdef complex cos_theta_outer = all_cos_theta[0]
    cdef np.ndarray [complex, ndim=1] cos_theta = all_cos_theta[1:M + 1]
    cdef complex cos_theta_substrate = all_cos_theta[M+1]

    cdef np.ndarray [complex, ndim=1] exps = np.exp(1j * (k_outer * d) * (n / n_outer) * cos_theta)

    cdef complex a_mat = -1
    cdef complex b_mat = 1
    cdef complex c_mat = 1
    cdef complex d_mat = n_substrate * cos_theta_substrate

    cdef int ind
    cdef list columns = [
                                                  np.array([
                                                      [0, exps[0]],
                                                      [1,
                                                       -(n[0] / n_outer) * (cos_theta[0] / cos_theta_outer) * exps[0]],
                                                      [(n[0] / n_outer) * (cos_theta[0] / cos_theta_outer), -1],
                                                      [-exps[0], n[0] * cos_theta[0]],
                                                      [-exps[0] * n[0] * cos_theta[0], 0]
                                                  ], dtype=np.complex_)
                                              ] + [
                                                  np.array([
                                                      [0, exps[ind]],
                                                      [1, -n[ind] * cos_theta[ind] * exps[ind]],
                                                      [n[ind] * cos_theta[ind], -1],
                                                      [-exps[ind], n[ind] * cos_theta[ind]],
                                                      [-exps[ind] * n[ind] * cos_theta[ind], 0]
                                                  ])
                                                  for ind in range(1, M)
                                              ]

    cdef np.ndarray [complex, ndim=2] mat = np.hstack([
        np.array([[0], [0], [a_mat], [b_mat], [0]], dtype=np.complex_),
        *columns,
        np.array([[0], [c_mat], [d_mat], [0], [0]], dtype=np.complex_)
    ])

    return mat

In [78]:
print(test_matrices_close(_make_matrix(*args), make_matrix_cython(*args)))

True


In [79]:
%timeit make_matrix_cython(*args)

151 µs ± 4.02 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Replacing np.concat of block matrices with C for loop

In [80]:
%%cython -a

cimport numpy as np
import numpy as np

cpdef np.ndarray [complex, ndim=2] make_matrix_cython2(int polarisation, int M, np.ndarray [double, ndim=1] n, np.ndarray [double, ndim=1] d, double k_outer, double n_outer, double n_substrate, double theta_outer):
    all_n = np.concatenate([np.array([n_outer]), n, np.array([n_substrate])])
    all_cos_theta = np.sqrt(np.complex_(1 - (n_outer / all_n) ** 2 * np.sin(theta_outer) ** 2))

    cdef complex cos_theta_outer = all_cos_theta[0]
    cdef np.ndarray [complex, ndim=1] cos_theta = all_cos_theta[1:M + 1]
    cdef complex cos_theta_substrate = all_cos_theta[M+1]

    cdef np.ndarray [complex, ndim=1] exps = np.exp(1j * (k_outer * d) * (n / n_outer) * cos_theta)

    cdef complex a_mat = -1
    cdef complex b_mat = 1
    cdef complex c_mat = 1
    cdef complex d_mat = n_substrate * cos_theta_substrate

    cdef np.ndarray [complex, ndim=2] mat = np.zeros((5, 2*M+2), dtype=complex)
    mat[2, 0] = a_mat
    mat[3, 0] = b_mat
    mat[1, 2 * M + 1] = c_mat
    mat[2, 2 * M + 1] = d_mat

    mat[0, 1] = 0
    mat[1, 1] = 1
    mat[2, 1] = (n[0] / n_outer) * (cos_theta[0] / cos_theta_outer)
    mat[3, 1] = -exps[0]
    mat[4, 1] = -exps[0] * n[0] * cos_theta[0]

    mat[0, 2] = exps[0]
    mat[1, 2] = -(n[0] / n_outer) * (cos_theta[0] / cos_theta_outer) * exps[0]
    mat[2, 2] = -1
    mat[3, 2] = n[0] * cos_theta[0]
    mat[4, 2] = 0

    cdef int i
    for i in range(1, M):
        mat[0, 2 * i + 1] = 0
        mat[1, 2 * i + 1] = 1
        mat[2, 2 * i + 1] = n[i] * cos_theta[i]
        mat[3, 2 * i + 1] = -exps[i]
        mat[4, 2 * i + 1] = -exps[i] * n[i] * cos_theta[i]

        mat[0, 2 * i + 2] = exps[i]
        mat[1, 2 * i + 2] = -n[i] * cos_theta[i] * exps[i]
        mat[2, 2 * i + 2] = -1
        mat[3, 2 * i + 2] = n[i] * cos_theta[i]
        mat[4, 2 * i + 2] = 0

    return mat

In [81]:
# make_matrix_cython2(*args)
print(test_matrices_close(_make_matrix(*args), make_matrix_cython2(*args)))

True


In [82]:
%timeit make_matrix_cython2(*args)

23.9 µs ± 348 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Replacing np.concat to create array of all n with C function calls

In [83]:
%%cython -a

cimport numpy as np
import numpy as np
import cython

cdef extern from "<complex.h>" nogil:
    double complex cexp(double complex z)
    double real(double complex z)
    double complex csqrt(double complex z)

cdef extern from "<math.h>" nogil:
    double cos(double arg)
    double sin(double arg)

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
@cython.cdivision(True) # No division by zero checking
cpdef np.ndarray [complex, ndim=2] make_matrix_cython3(int polarisation, int M, np.ndarray [double, ndim=1] n, np.ndarray [double, ndim=1] d, double k_outer, double n_outer, double n_substrate, double theta_outer):
    cdef complex cos_theta_outer = cos(theta_outer)
    cdef np.ndarray [complex, ndim=1] cos_theta = np.sqrt(np.complex_(1 - (n_outer / n) ** 2 * np.sin(theta_outer) ** 2))
    cdef complex cos_theta_substrate = csqrt(1 - (n_outer / n_substrate) ** 2 * sin(theta_outer) ** 2)

    cdef np.ndarray [complex, ndim=1] exps = np.exp(1j * (k_outer * d) * (n / n_outer) * cos_theta)

    cdef complex a_mat = -1
    cdef complex b_mat = 1
    cdef complex c_mat = 1
    cdef complex d_mat = n_substrate * cos_theta_substrate

    cdef np.ndarray [complex, ndim=2] mat = np.zeros((5, 2*M+2), dtype=complex)
    mat[2, 0] = a_mat
    mat[3, 0] = b_mat
    mat[1, 2 * M + 1] = c_mat
    mat[2, 2 * M + 1] = d_mat

    mat[0, 1] = 0
    mat[1, 1] = 1
    mat[2, 1] = (n[0] / n_outer) * (cos_theta[0] / cos_theta_outer)
    mat[3, 1] = -exps[0]
    mat[4, 1] = -exps[0] * n[0] * cos_theta[0]

    mat[0, 2] = exps[0]
    mat[1, 2] = -(n[0] / n_outer) * (cos_theta[0] / cos_theta_outer) * exps[0]
    mat[2, 2] = -1
    mat[3, 2] = n[0] * cos_theta[0]
    mat[4, 2] = 0

    cdef int i
    for i in range(1, M):
        mat[0, 2 * i + 1] = 0
        mat[1, 2 * i + 1] = 1
        mat[2, 2 * i + 1] = n[i] * cos_theta[i]
        mat[3, 2 * i + 1] = -exps[i]
        mat[4, 2 * i + 1] = -exps[i] * n[i] * cos_theta[i]

        mat[0, 2 * i + 2] = exps[i]
        mat[1, 2 * i + 2] = -n[i] * cos_theta[i] * exps[i]
        mat[2, 2 * i + 2] = -1
        mat[3, 2 * i + 2] = n[i] * cos_theta[i]
        mat[4, 2 * i + 2] = 0

    return mat

In [84]:
# make_matrix_cython2(*args)
print(test_matrices_close(_make_matrix(*args), make_matrix_cython3(*args)))

True


In [85]:
%timeit make_matrix_cython3(*args)

19.6 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Test function for parallelisation

In [86]:
%%cython -a

from cython.parallel import prange

cpdef int test():
    cdef int sum
    cdef int i
    sum = 0
    for i in prange(10, nogil=True) :
        sum += i
    return sum

In [87]:
test()

45

# Parallelisation of for loop: Using memoryview and nogil=True

In [88]:
%%cython -a

cimport numpy as np
import numpy as np
import cython
from cython.parallel import prange

cdef extern from "<complex.h>" nogil:
    double complex cexp(double complex z)
    double real(double complex z)
    double complex csqrt(double complex z)

cdef extern from "<math.h>" nogil:
    double cos(double arg)
    double sin(double arg)

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
@cython.cdivision(True) # No division by zero checking
cpdef np.ndarray [complex, ndim=2] make_matrix_cython4(int polarisation, int M, np.ndarray [double, ndim=1] n, np.ndarray [double, ndim=1] d, double k_outer, double n_outer, double n_substrate, double theta_outer):
    cdef complex cos_theta_outer = cos(theta_outer)
    cdef np.ndarray [complex, ndim=1] cos_theta = np.sqrt(np.complex_(1 - (n_outer / n) ** 2 * np.sin(theta_outer) ** 2))
    cdef complex cos_theta_substrate = csqrt(1 - (n_outer / n_substrate) ** 2 * sin(theta_outer) ** 2)

    cdef np.ndarray [complex, ndim=1] exps = np.exp(1j * (k_outer * d) * (n / n_outer) * cos_theta)

    cdef complex a_mat = -1
    cdef complex b_mat = 1
    cdef complex c_mat = 1
    cdef complex d_mat = n_substrate * cos_theta_substrate

    cdef complex [:, :] mat = np.zeros((5, 2*M+2), dtype=complex)
    mat[2, 0] = a_mat
    mat[3, 0] = b_mat
    mat[1, 2 * M + 1] = c_mat
    mat[2, 2 * M + 1] = d_mat

    mat[0, 1] = 0
    mat[1, 1] = 1
    mat[2, 1] = (n[0] / n_outer) * (cos_theta[0] / cos_theta_outer)
    mat[3, 1] = -exps[0]
    mat[4, 1] = -exps[0] * n[0] * cos_theta[0]

    mat[0, 2] = exps[0]
    mat[1, 2] = -(n[0] / n_outer) * (cos_theta[0] / cos_theta_outer) * exps[0]
    mat[2, 2] = -1
    mat[3, 2] = n[0] * cos_theta[0]
    mat[4, 2] = 0

    cdef int i
    for i in prange(1, M, nogil=True):
        mat[0, 2 * i + 1] = 0
        mat[1, 2 * i + 1] = 1
        mat[2, 2 * i + 1] = n[i] * cos_theta[i]
        mat[3, 2 * i + 1] = -exps[i]
        mat[4, 2 * i + 1] = -exps[i] * n[i] * cos_theta[i]

        mat[0, 2 * i + 2] = exps[i]
        mat[1, 2 * i + 2] = -n[i] * cos_theta[i] * exps[i]
        mat[2, 2 * i + 2] = -1
        mat[3, 2 * i + 2] = n[i] * cos_theta[i]
        mat[4, 2 * i + 2] = 0

    return np.asarray(mat)

In [89]:
test_matrices_close(_make_matrix(*args), make_matrix_cython4(*args))

True

In [90]:
%timeit make_matrix_cython4(*args)

31.6 µs ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Results

Winner: make_matrix_cython3 (maximal cythonisation but no parallelisation)